In [1]:
import os, sys
import pandas as pd
import numpy as np
import json

In [2]:
# path 설정
sys.path.append((os.path.abspath("")))
print(sys.path[-1])

d:\base\boostcamp\temp\code


In [4]:
data_path: str = "data/eval/ens"
ease: pd.DataFrame = pd.read_csv(os.path.join(data_path,"submission-ease.csv"))
recvae: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recvae_100_score.csv"))
bert: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_bert4rec_top20_score.csv"))
lgcn: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_lightgcn_1126_top20_score.csv"))
admm: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_ADMMSLIM_100_top20_score.csv"))
autoint: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_autoint_top20_score.csv"))
bpr: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_bpr_top20_score.csv"))
gru: pd.DataFrame = pd.read_csv(os.path.join(data_path,"recbole_gru4recf_top20_score.csv"))

In [5]:
# 각 데이터프레임에 대해 score를 표준화하여 새로운 열 추가
ease['score_normalized'] = ease.groupby('user')['score'].transform(lambda x: (x - x.mean()) / x.std())
recvae['score_normalized'] = recvae.groupby('user')['score'].transform(lambda x: (x - x.mean()) / x.std())
bert['score_normalized'] = bert.groupby('user')['score'].transform(lambda x: (x - x.mean()) / x.std())
lgcn['score_normalized'] = lgcn.groupby('user')['score'].transform(lambda x: (x - x.mean()) / x.std())
admm['score_normalized'] = admm.groupby('user')['score'].transform(lambda x: (x - x.mean()) / x.std())
autoint['score_normalized'] = autoint.groupby('user')['score'].transform(lambda x: (x - x.mean()) / x.std())
bpr['score_normalized'] = bpr.groupby('user')['score'].transform(lambda x: (x - x.mean()) / x.std())
gru['score_normalized'] = gru.groupby('user')['score'].transform(lambda x: (x - x.mean()) / x.std())


In [6]:
gru

,Unnamed: 0,user,item,score,score_normalized
0,0,63312,5957,8.637482,2.491660
1,1,63312,5308,8.249784,1.787991
2,2,63312,4992,8.225126,1.743238
3,3,63312,6535,7.749802,0.880528
4,4,63312,5313,7.461710,0.357643
...,...,...,...,...,...
627195,627195,54599,55444,6.528383,-0.898924
627196,627196,54599,74545,6.526614,-0.902709
627197,627197,54599,105844,6.503645,-0.951840
627198,627198,54599,106782,6.493541,-0.973453


In [16]:
# 세 모델의 정규화된 점수를 합산
merged_df = pd.concat([
    ease[['user', 'item', 'score_normalized']],
    admm[['user', 'item', 'score_normalized']], 
    gru[['user', 'item', 'score_normalized']],
    bert[['user', 'item', 'score_normalized']],
])

# 유저-아이템 쌍별로 정규화된 점수 합산
ensemble_df = merged_df.groupby(['user', 'item'])['score_normalized'].sum().reset_index()

# 유저별로 상위 10개 아이템 선택
final_df = ensemble_df.sort_values(['user', 'score_normalized'], ascending=[True, False]) \
    .groupby('user').head(10) \
    .reset_index(drop=True)

final_df

,user,item,score_normalized
0,11,4886,3.679627
1,11,4370,3.292482
2,11,40815,3.148030
3,11,30707,2.289479
4,11,63876,2.138322
...,...,...,...
313595,138493,5349,1.800425
313596,138493,110,1.528498
313597,138493,45499,1.368196
313598,138493,68358,1.351455


In [17]:
submission = final_df.loc[:, ['user', 'item']]
submission


,user,item
0,11,4886
1,11,4370
2,11,40815
3,11,30707
4,11,63876
...,...,...
313595,138493,5349
313596,138493,110
313597,138493,45499
313598,138493,68358


In [18]:
submission.to_csv("data/eval/ens/submission/ens_soft_esg4.csv", index=False)
